### Importamos las librerias

In [205]:
import ast
import pandas as pd
from fastapi import FastAPI 

### Accedemos al Dataset

In [206]:
games = []
with open('steam_games.json') as f: #permite acceder al contenido del archivo JSON
    for line in f.readlines():
        games.append(ast.literal_eval(line)) #convirtir cada línea del archivo JSON en un diccionario de Python y añadiéndolo a la lista games

df = pd.DataFrame(games) # carga los datos en un DataFrame de pandas

In [209]:
# Cambiar el dtype de la columna'release_date' ya que contiene fechas en formato de texto
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

In [210]:
#Cambiar el dtype de la columna 'metascore' ya que contiene puntajes en formato de texto
df['metascore'] = pd.to_numeric(df['metascore'], errors='coerce')

### Creamos las funciones para la API

In [184]:
# Indicamos título y descripción de la API
app = FastAPI(title='PI N°1 (MLOps) -Angie Arango Zapata DTS13')

In [185]:
# Endpoint 1: Géneros más repetidos por año
@app.get('/ Genero')
def genero(año: str):
    # Filtrar el DataFrame por el año proporcionado
    df_filtrado = df[df['release_date'].dt.year == int(año)]
    
    # Obtener una lista con todos los géneros para el año dado
    generos_list = df_filtrado['genres'].explode().tolist()

    # Calcular el conteo de cada género en la lista
    generos_count = pd.Series(generos_list).value_counts()

    # Obtener los 5 géneros más vendidos en orden correspondiente
    generos_mas_repetidos = generos_count.nlargest(5).to_dict()

    return generos_mas_repetidos

In [192]:
# Endpoint 2: Juegos lanzados en un año
@app.get('/ Juegos')
def juegos(año: str):
    # Filtra los datos por el año especificado
    df_filtrado = df[df['release_date'].dt.year == int(año)]

    # Obtiene la lista de juegos lanzados en el año
    juegos_lanzados = df_filtrado['title'].tolist()

    return juegos_lanzados

In [187]:
# Endpoint 3: Specs más comunes por año
@app.get('/ Specs')
def specs(año: str):
    # Filtrar el DataFrame por el año proporcionado
    df_filtrado = df[df['release_date'].dt.year == int(año)]
    
    # Obtener una lista con todas las especificaciones para el año dado
    specs_list = df_filtrado['specs'].explode().tolist()

    # Calcular el conteo de cada especificación en la lista
    specs_count = pd.Series(specs_list).value_counts()

    # Obtener las 5 especificaciones más comunes en orden correspondiente
    top_5_specs = specs_count.nlargest(5).to_dict()

    return top_5_specs

In [188]:
# Endpoint 4: Cantidad de juegos con early access en un año
@app.get('/ Earlyaccess')
def earlyaccess(año: str):
    #Filtrar los datos por el año especificado y por juegos con early access
    df_filtrado = df[(df['release_date'].dt.year == int(año)) & (df['early_access'] == True)]

    #Contar la cantidad de juegos con early access en el año especificado
    cantidad_juegos_early_access = len(df_filtrado)

    return cantidad_juegos_early_access

In [189]:
# Endpoint 5: Análisis de sentimiento por año
@app.get('/ Sentimient')
def sentiment(año: str):
    # Filtrar los datos por el año especificado
    df_filtrado = df[df['release_date'].dt.year == int(año)]

    # Contar la cantidad de registros que cumplen con cada análisis de sentimiento
    sentimient_counts = df_filtrado['sentiment'].value_counts()

    # Convertir la serie de conteo en un diccionario
    sentimient_dict = sentimient_counts.to_dict()

    # Eliminar sentimientos que no están en la lista mencionada
    sentimient_valid = ["Mixed", "Positive", "Very Positive", "Mostly Positive",
                            "Negative", "Very Negative", "Mostly Negative", "Overwhelmingly Positive"]
    sentimient_dict = {sentimient: count for sentimient, count in sentimient_dict.items() if sentimient in sentimient_valid}

     # Verificar si el diccionario está vacío
    if not sentimient_dict:
        return "Sin registros"

    return sentimient_dict

In [190]:
# Endpoint 6: Top 5 juegos con mayor metascore por año
@app.get('/ Metascore')
def metascore(año: str):
    #Filtrar los datos por el año especificado y ordena por metascore de forma descendente
    df_filtrado = df[df['release_date'].dt.year == int(año)].sort_values(by='metascore', ascending=False)

    #Seleccionar los 5 juegos con mayor metascore y obtiene su información
    top_juegos_metascore = df_filtrado.head(5)[['title', 'metascore']].to_dict(orient='records')

    return top_juegos_metascore